# Differences that pre-processing cause to the baseline model in Semeval 2010 data

for reference, command that was run within scripts/ was ```CUDA_VISIBLE_DEVICES=<device_no> python main.py --<cross_validate/use_test> --dataset=ddi --preprocessing_type=<entity_blinding/punct_digit/punct_stop_digit> --border_size=-1 --num_epoches=100 --lr_values 0.001 0.0001 --lr_boundaries 60```
This was gotten after preliminary hyperparameter tuning

In [1]:
from scipy.stats import ttest_rel
def paired_ttest(score1, score2):
    all_three_macroF1_score1 = [x for x in zip(*score1)]
    all_three_macroF1_score2 = [x for x in zip(*score2)]
    ttests = [ttest_rel(macro_f1_score1, macro_f1_score2) 
            for macro_f1_score1, macro_f1_score2 in zip(all_three_macroF1_score1, all_three_macroF1_score2)]
    print('5 way evaluation with none: \t', ttests[0])
    print('5 way evaluation without none: \t', ttests[1])
    print('2 way evaluation: \t\t', ttests[2])

## First compare the cross validated score differences

In [10]:
# below is the result with -1 border size (newer implementation)
baseline_test = (65.53, 58.41, 81.74)
# model ID 8147a3f3-3c74-4194-9fa4-4de9bd21b6ff


# baseline_test = (66.66, 59.73, 82.6) # Model ID 92d0466a-e8c0-4e72-8068-f10f699d8824 on harrison
# Results are 5 way with none, 5 way without none, 2 way (just detection of DDI) from Jan 23 2019 run

In [11]:
baseline = [(80.05, 76.15, 85.99), (79.73, 75.64, 87.54), 
            (78.13, 73.77, 86.07), (78.48, 74.18, 86.03), (80.63, 76.83, 86.9)]
# model ID 152b5ba9-100e-4024-93c0-40976064e8e0
# 79.40 +- 0.95 75.31 +- 1.16 86.51 +- 0.62

# old baseline number (still relevant)
# Still need to run this baseline = but refer to old baseline for now from Nov 26, 2018
#baseline = currently running on harrison Feb 15, 2019
# baseline = [(78.82, 74.65, 85.68), (79.89, 75.84, 87.65), 
#             (78.98, 74.84, 86.07), (79.04, 74.86, 86.29), (81.0, 77.23, 87.58)]
# model ID e16e6184-d403-480e-b718-e08e2f4a3c5f on harrison

In [12]:
# paired_ttest(baseline, baseline_new) # they're pretty much the same

In [13]:
# newer implementation with border size -1
entity_blinding = [(79.24, 75.11, 86.84), (80.4, 76.48, 87.5), (81.9, 78.21, 88.74), 
                   (83.43, 80.12, 89.22), (79.67, 75.53, 88.04)]
# model ID b631a6c5-f713-4dc6-9335-c4a268cec02a
# 80.93 +- 1.54 77.09 +- 1.85 88.07 +- 0.85

# entity_blinding = [(79.41, 75.3, 86.97), (79.22, 75.09, 86.61), 
#                    (81.29, 77.48, 88.36), (83.06, 79.71, 88.9), (80.83, 76.93, 88.41)]
# model ID dfafa161-186b-4b73-a0ec-50cb40f2f3f9 on harrison

In [14]:
punct_digit = [(77.42, 72.98, 85.14), (76.77, 72.21, 84.58), 
               (77.98, 73.51, 86.38), (81.65, 78.03, 87.67), (76.36, 71.63, 85.48)]
# model ID b24f39ec-3672-46a4-8d32-60fef0e8938e on harrison

In [15]:
punct_stop_digit = [(75.46, 70.66, 83.17), (74.44, 69.34, 83.08), 
                    (76.36, 71.62, 84.59), (78.9, 74.75, 85.52), (74.65, 69.61, 83.21)]
# model ID 82691fa7-4ecc-48a7-a16f-211b5fdd951a on harrison

In [16]:
ner_blinding = [(75.93, 71.0, 86.31), (76.43, 71.77, 84.46), 
                (73.81, 68.44, 84.41), (76.95, 72.34, 85.41), (75.71, 70.81, 85.46)]
# model ID 42f73da2-45d9-4a62-8084-425fe18fe1ac on gray

In [17]:
paired_ttest(baseline, entity_blinding)

5 way evaluation with none: 	 Ttest_relResult(statistic=-1.2628049701194481, pvalue=0.2752585521881439)
5 way evaluation without none: 	 Ttest_relResult(statistic=-1.2154192817578147, pvalue=0.29103541643971315)
2 way evaluation: 		 Ttest_relResult(statistic=-2.6161768589692382, pvalue=0.059035760430435284)


In [18]:
paired_ttest(baseline, punct_digit)

5 way evaluation with none: 	 Ttest_relResult(statistic=1.039685386416001, pvalue=0.35719925259456997)
5 way evaluation without none: 	 Ttest_relResult(statistic=1.0357706531069237, pvalue=0.35881703992003744)
2 way evaluation: 		 Ttest_relResult(statistic=0.8421850664931626, pvalue=0.4470931112150066)


In [19]:
paired_ttest(baseline, punct_stop_digit)

5 way evaluation with none: 	 Ttest_relResult(statistic=2.864105930010297, pvalue=0.045739598124896454)
5 way evaluation without none: 	 Ttest_relResult(statistic=2.8377644118074516, pvalue=0.0469738769523503)
2 way evaluation: 		 Ttest_relResult(statistic=3.608663115062657, pvalue=0.0225834258947458)


In [20]:
paired_ttest(punct_digit, punct_stop_digit)

5 way evaluation with none: 	 Ttest_relResult(statistic=9.920606445770595, pvalue=0.0005796131833806519)
5 way evaluation without none: 	 Ttest_relResult(statistic=9.435137013976645, pvalue=0.000703582899732976)
2 way evaluation: 		 Ttest_relResult(statistic=14.235294117646847, pvalue=0.0001414266519424943)


In [21]:
paired_ttest(baseline, ner_blinding)

5 way evaluation with none: 	 Ttest_relResult(statistic=6.194245259556999, pvalue=0.003453572633737003)
5 way evaluation without none: 	 Ttest_relResult(statistic=6.023229418698782, pvalue=0.0038279203001210784)
2 way evaluation: 		 Ttest_relResult(statistic=2.29025314100239, pvalue=0.08383370074849063)


In short, entity blinding doesn't cause a statistically siginficant difference for the DDI dataset, maybe enough information is available in the sentence itself. Removing punctuations and normalizing digits has no effect, but removing stop words is harmful. NER blinding is harmful (because there is no stratification of types so we are losing useful information presumably. 

Test score results for the above are: (border size -1 with newer implementation)

```(65.53, 58.41, 81.74)``` for baseline model with ID 8147a3f3-3c74-4194-9fa4-4de9bd21b6ff

```(67.02, 60.2, 82.37)``` for entity blinding model ID b8cc9fde-1386-4319-86ef-c4cad47ff5f2 

```(63.41, 55.8, 80.49)``` for punctuation removal and digit normalization with model ID 29f06a86-f050-4a10-9201-fb5e3053caba

```(55.87, 46.63, 76.57)``` for punctuation and stop word removal and digit normalization with model ID dc47d4d7-7bef-4172-b5bc-6425a50a5512

```(57.22, 48.21, 79.03)``` for ner blinding with model ID d6a19021-3b95-47f0-8ba0-69390ef105b2

In [8]:
# Test score results for the above are (all model IDs on harrison): (old results with -1 border size)

# ```(66.66, 59.73, 82.6)``` for baseline model with ID 92d0466a-e8c0-4e72-8068-f10f699d8824

# ```(66.15, 59.1, 82.62)``` for entity blinding model ID 851b71a8-bea3-4b0d-8024-16ea836ab0ad

# ```(63.43, 55.88, 80.02)``` for punctuation removal and digit normalization with model ID 9a50733f-8644-431b-b50b-ef78b3754144

# ```(58.48, 49.93, 76.56)``` for punctuation and stop word removal and digit normalization with model ID 4ba070e0-324a-48cf-934e-005ddaf614b8

# ```(59.66, 51.28, 79.14)``` for ner blinding with model ID 374e9f6c-b6f8-4f4e-ac80-cf6300eac0c2 on gray